# Config

In [451]:
sales_order_file = r"C:\Users\Admin\OneDrive - neousys-tech\Share NTA Warehouse\Daily Update\Open Sales Order 8_26_2025.CSV"
warehouse_inventory_file = r"C:\Users\Admin\OneDrive - neousys-tech\Share NTA Warehouse\Daily Update\WH01S_8_25.CSV"
shipping_schedule_file = r"C:\Users\Admin\OneDrive - neousys-tech\Share NTA Warehouse\Daily Update\NTA_Shipping schedule_20250529.xlsx"

In [452]:
import os
import logging
from flask import Flask, jsonify # type: ignore
from flask_sqlalchemy import SQLAlchemy # type: ignore
from docx import Document # type: ignore
import pdfplumber # type: ignore
import pandas as pd # type: ignore
import json
import sys
import requests
import re
from docx import Document
from datetime import datetime, timedelta
from sqlalchemy import Column, String, Float, Date, Integer
import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
import numpy as np
import ipywidgets as widgets
from IPython.display import display

In [453]:
## === Configure Supabase === ##
logging.basicConfig(level=logging.INFO)

app = Flask(__name__)

app.config['SQLALCHEMY_DATABASE_URI'] = (
    "postgresql://postgres.avcznjglmqhmzqtsrlfg:"
    "Czheyuan0227@aws-0-us-east-2.pooler.supabase.com:6543/postgres"
)
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False


db = SQLAlchemy()

db.init_app(app)

class PDFFileLog(db.Model):
    __tablename__ = "pdf_file_log"   # must match your table name in Postgres

    order_id = db.Column(db.String, primary_key=True)   # adjust type/PK to match schema
    file_name = db.Column(db.String)
    file_path = db.Column(db.String)
    extracted_data = db.Column(db.JSON)  # if it’s a JSON column



## === Models Defination === ##

def write_df_to_gsheet(df, sheet_name, worksheet_name, cred_path="path/to/your/credentials.json"):
    scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials.from_json_keyfile_name(cred_path, scope)
    client = gspread.authorize(creds)

    sheet = client.open(sheet_name)
    try:
        worksheet = sheet.worksheet(worksheet_name)
        worksheet.clear()
    except gspread.exceptions.WorksheetNotFound:
        worksheet = sheet.add_worksheet(title=worksheet_name, rows=1000, cols=20)

    set_with_dataframe(worksheet, df)
    print(f"✅ Data written to Google Sheet: {sheet_name} [{worksheet_name}]")

def components_for_wo(df, wo):
    # wo_norm = re.sub(r'^SO-', '', str(wo)).strip()
    wo_norm = wo
    return df[df['WO'] == wo_norm].sort_values('Product Number').reset_index(drop=True)


def normalize_wo_number(wo):
    # Extract the 8-digit number starting with year, E.g. "SO-20250975"
    match = re.search(r'\b(20\d{6})\b', wo)
    if match:
        return f"SO-{match.group(1)}"
    return wo  # return original if no match



## reorder parts in sales order, follow pdf structure(Sales Order should follow the PDF’s sequence rather than alphabetical order)
def reorder_df_out_by_output(output_df: pd.DataFrame, df_out: pd.DataFrame) -> pd.DataFrame:
    # 1) output_df is from PDFs, df_out is from Excel
    ref = output_df.copy()
    ref['__pos_out'] = ref.groupby('WO').cumcount()                 # position within WO
    ref['__occ'] = ref.groupby(['WO','Product Number']).cumcount()  # occurrence index for duplicates
    ref_key = ref[['WO','Product Number','__occ','__pos_out']]

    # 2) On df_out, tag each duplicate with its own occurrence index
    tgt = df_out.copy()
    tgt['__occ'] = tgt.groupby(['WO','Product Number']).cumcount()

    # 3) Merge positions from output_df to df_out rows (match by WO + Product + occurrence)
    merged = tgt.merge(ref_key, on=['WO','Product Number','__occ'], how='left')

    # 4) For rows not present in output_df, keep their original within-WO order but push them after the matched ones
    merged['__fallback'] = merged.groupby('WO').cumcount()
    merged['__pos_out'] = merged['__pos_out'].fillna(np.inf)

    # 5) Final order: by WO, then by output_df position; if missing, by original order
    ordered = (merged
               .sort_values(['WO','__pos_out','__fallback'])
               .drop(columns=['__occ','__pos_out','__fallback'])
               .reset_index(drop=True))
    return ordered



def extracted_to_df(order):
    """Return a 2‑column DataFrame: ['WO','Product Number'] for one PDFFileLog row."""
    if order is None:
        return pd.DataFrame(columns=["WO", "Product Number"])

    data = order.extracted_data or {}
    if isinstance(data, str):              # if stored as TEXT/JSONB string
        try:
            data = json.loads(data)
        except Exception:
            data = {}

    items = data.get("items") or []
    wo = data.get("wo") or getattr(order, "order_id", "")

    rows = [{
        "WO": wo,
        "Product Number": (
            it.get("product_number") or it.get("part_number")
            or it.get("product") or it.get("part") or ""
        ),
    } for it in items]

    if not rows:
        rows = [{"WO": wo, "Product Number": ""}]

    return pd.DataFrame(rows, columns=["WO", "Product Number"])

In [454]:
# ## Parse PDF then upload to google sheet, highlight when there's a dash in file
# def has_dash_comments(pdf_path):
#     with pdfplumber.open(pdf_path) as pdf:
#         for page in pdf.pages:
#             if not hasattr(page, "annots") or page.annots is None:
#                 continue
#             for annot in page.annots:
#                 content = annot.get("contents") or ""
#                 if "-" in content:
#                     return True
#     return False

# def extract_pdf_data_with_dash_check(pdf_path):
#     filename = os.path.basename(pdf_path).replace('.pdf', '')

#     # Extract WO
#     wo_match = re.search(r"WO-(\d{8}(?:r\d*)?)", filename)
#     wo = wo_match.group(1) if wo_match else None

#     # Extract PO
#     po_match = re.search(r"(PO[#_])(\w+)", filename)
#     po = po_match.group(2) if po_match else None

#     # Extract customer
#     customer = None
#     if wo_match and po_match:
#         try:
#             customer_raw = filename.split(f"WO-{wo}_")[1].split(po_match.group(0))[0]
#             customer = customer_raw.strip('_ ').replace('_', ' ')
#         except Exception:
#             customer = None

#     dash_flag = has_dash_comments(pdf_path)

#     if not all([wo, customer, po]):
#         return wo, customer, po, pd.DataFrame(), dash_flag

#     # Extract tables
#     tables = []
#     cleaned_header = ['Item', 'Description', 'Site', 'Ordered', 'To Pick']

#     with pdfplumber.open(pdf_path) as pdf:
#         for page in pdf.pages:
#             table = page.extract_table()
#             if not table:
#                 continue
#             header = table[0]
#             cleaned_raw_header = [str(col).strip() if col else "" for col in header]

#             if cleaned_raw_header == cleaned_header:
#                 df = pd.DataFrame(table[1:], columns=cleaned_header)
#                 tables.append(df)
#             elif 'P.O. No.' in cleaned_raw_header:
#                 try:
#                     po_index = cleaned_raw_header.index('P.O. No.')
#                     data_rows = [row[:po_index] + row[po_index+1:] for row in table[1:]]
#                     cleaned_alt_header = cleaned_raw_header[:po_index] + cleaned_raw_header[po_index+1:]
#                     if len(cleaned_header) == len(cleaned_alt_header):
#                         df = pd.DataFrame(data_rows, columns=cleaned_header)
#                         tables.append(df)
#                 except:
#                     continue

#     if not tables:
#         return wo, customer, po, pd.DataFrame(), dash_flag

#     # Clean
#     final_df = pd.concat(tables, ignore_index=True)
#     final_df.columns = final_df.columns.str.strip()
#     final_df = final_df[final_df['Site'].astype(str).str.startswith('WH')].copy()

#     def extract_or_default(row):
#         col1 = row['Item']
#         col2 = row['Description']

#         def clean(text):
#             return (
#                 text.strip()
#                     .lstrip('*')
#                     .replace("(cid:95)", "_")
#                     .replace("(cid:74)", "J")
#             )

#         if isinstance(col1, str) and '...' in col1:
#             if isinstance(col2, str):
#                 part = col2.strip().split('\n')[0].strip()
#                 return clean(part)
#             return None
#         elif isinstance(col1, str):
#             return clean(col1)
#         else:
#             return None

#     final_df['Extracted'] = final_df.apply(extract_or_default, axis=1)
#     return wo, customer, po, final_df[['Extracted', 'Ordered']], dash_flag

# def process_all_pdfs(folder_path):
#     all_rows = []

#     for root, _, files in os.walk(folder_path):
#         for file in files:
#             if file.lower().endswith(".pdf"):
#                 pdf_path = os.path.join(root, file)
#                 wo, customer, po, df, dash_flag = extract_pdf_data_with_dash_check(pdf_path)
#                 if df.empty:
#                     continue
#                 for _, row in df.iterrows():
#                     all_rows.append({
#                         "Customer": customer,
#                         "PO": po,
#                         "WO": f"SO-{wo}",
#                         "Product Number": row['Extracted'],
#                         "Qty": row['Ordered'],
#                         "Source File": file,
#                         "Flagged": "⚠️" if dash_flag else ""
#                     })

#     return pd.DataFrame(all_rows)



# # Parse WO PDFs and write to Google Sheet
# folder = r"\\Quickbook2024\d\Drive D\QuickBooks\3- Year 2025\Work Order- WO"
# google_cred_path = r"C:\Users\Admin\Downloads\pdfwo-466115-734096e1cef8.json"

# output_df = process_all_pdfs(folder)

# if not output_df.empty:
#     write_df_to_gsheet(
#         output_df,
#         sheet_name="PDF_WO",
#         worksheet_name="Sheet1",
#         cred_path=google_cred_path
#     )
# else:
#     print("❌ No output to write.")


## Ingest

In [455]:
df_sales_order = pd.read_csv(sales_order_file, encoding="ISO-8859-1")
inventory_df = pd.read_csv(warehouse_inventory_file)
df_shipping_schedule = pd.read_excel(shipping_schedule_file)



# === Fetch API: Picked Status ===
url = "http://192.168.60.133:5001/api/word-files"
response = requests.get(url)

if response.status_code == 200:
    api_data = response.json()
    if "word_files" in api_data and isinstance(api_data["word_files"], list):
        word_files_df = pd.DataFrame(api_data["word_files"])
    else:
        word_files_df = pd.DataFrame(columns=["file_name", "order_id", "status"])
else:
    word_files_df = pd.DataFrame(columns=["file_name", "order_id", "status"])

word_files_df.rename(columns={
    'order_id': 'WO_Number'
}, inplace=True)

print(
    f"sales order: {df_sales_order.columns.tolist()}",
    f"inventory: {inventory_df.columns.tolist()}",
    f"shipping schedule: {df_shipping_schedule.columns.tolist()}",
    f"word files: {word_files_df.columns.tolist()}",
    sep="\n"
)


sales order: ['Unnamed: 0', 'Type', 'Date', 'Ship Date', 'Deliv Date', 'Terms', 'Due Date', 'Num', 'P. O. #', 'Name', 'Qty', 'Invoiced', 'Backordered', 'Amount', 'Item', 'Rep', 'Open Balance', 'Inventory Site']
inventory: ['Unnamed: 0', 'Reorder Pt (Min)', 'Max', 'On Hand', 'On Sales Order', 'Available', 'Order', 'On PO', 'Reorder Qty', 'Next Deliv', 'Sales/Week']
shipping schedule: ['Date', 'SO NO.', 'Ship to', 'Customer PO No.', 'Reference', 'Project Code', 'Model Name', 'Assemble Option', 'Qty', 'Remark', 'Ship Date', 'Description']
word files: ['file_name', 'file_path', 'WO_Number', 'status']


## Normalize

In [456]:
# Some Part Names are too long to show in QB
mappings = {
    'M.280-SSD-256GB-PCIe44-TLC5WT-T': 'M.280-SSD-256GB-PCIe44-TLC5WT-TD',
    'M.280-SSD-512GB-PCIe44-TLC5WT-T': 'M.280-SSD-512GB-PCIe44-TLC5WT-TD',
    'M.242-SSD-256GB-PCIe34-TLC5WT-T': 'M.242-SSD-256GB-PCIe34-TLC5WT-TD',
    'M.242-SSD-512GB-PCIe34-TLC5WT-T': 'M.242-SSD-512GB-PCIe34-TLC5WT-TD',
    'M.242-SSD-128GB-PCIe34-TLC5WT-T': 'M.242-SSD-128GB-PCIe34-TLC5WT-TD',
    'Cblkit-FP-NRU-230V-AWP_NRU-240S': 'Cblkit-FP-NRU-230V-AWP_NRU-240S-AWP',
}

### I. Open Sales Order

In [457]:
# === Normalize Open Sales Order ===
df_sales_order.rename(columns={
    'Unnamed: 0': 'Component',
    'Num': 'WO_Number',
    'Qty': 'Required_Qty'
}, inplace=True)


df_sales_order['Component'] = df_sales_order['Component'].ffill().astype(str).str.strip()
df_sales_order = df_sales_order[~df_sales_order["Component"].str.startswith("total")]
# df_sales_order["WO_Number"] = (
#     df_sales_order["WO_Number"].astype(str).str.strip().str.replace(r'^SO-', '', regex=True)
# )  ## remove "SO-", e.g. "20250975"
df_sales_order = df_sales_order[
    ~df_sales_order["Component"].str.lower().isin(["forwarding charge", "tariff (estimation)"])
]

df_sales_order = df_sales_order[df_sales_order['Inventory Site'] == 'WH01S-NTA']

word_files_df["WO_Number"] = word_files_df["WO_Number"].apply(normalize_wo_number)



### II. Inventory Status

In [458]:
# === Load Inventory File ===
inventory_df.rename(columns={'Unnamed: 0': 'Part_Number', 'Available': 'Stock_Available'}, inplace=True)
inventory_df["Part_Number"] = inventory_df["Part_Number"].astype(str).str.strip()

inventory_df['Part_Number'] = inventory_df['Part_Number'].replace(mappings)

In [459]:
df_sales_order.head(3)

,Component,Type,Date,Ship Date,Deliv Date,Terms,Due Date,WO_Number,P. O. #,Name,Required_Qty,Invoiced,Backordered,Amount,Item,Rep,Open Balance,Inventory Site
9,AccsyBx-Cardholder-10108GC-5080,Sales Order,07/18/2025,09/05/2025,NaN,Prepay,07/18/2025,SO-20251016,25-0271,"Four Growers, Inc.",10.0,0.0,10.0,200.0,Accessory:AccsyBx-Cardholder-10108GC-5080,A702,200.0,WH01S-NTA
10,AccsyBx-Cardholder-10108GC-5080,Sales Order,07/28/2025,09/12/2025,NaN,Prepay,07/28/2025,SO-20251073,1072,"Pattern Labs, Inc.",7.0,0.0,7.0,140.0,Accessory:AccsyBx-Cardholder-10108GC-5080,A800,140.0,WH01S-NTA
11,AccsyBx-Cardholder-10108GC-5080,Sales Order,08/21/2025,12/31/2025,NaN,Net 30,09/20/2025,SO-20251186,3008881458,The University of Michigan,1.0,0.0,1.0,20.0,Accessory:AccsyBx-Cardholder-10108GC-5080,A702,20.0,WH01S-NTA


### Update Google Sheet: Open Sales Order, PDF Sales Order

In [460]:
## Pull out Sales Order from Supabase
with app.app_context():
    orders = PDFFileLog.query.all()
    out = pd.concat([extracted_to_df(o) for o in orders], ignore_index=True)
    print(out)

output_df = out

# === Reorder Open Sales Order ===
needed_cols = {
    'Name': 'Customer',
    'P. O. #': 'Customer PO',
    'WO_Number': 'WO',
    'Component': 'Product Number',
    'Required_Qty': 'Qty',
    'Ship Date': 'Lead Time'
}
for c in ['Customer', 'PO']:
    if c not in df_sales_order.columns:
        df_sales_order[c] = ""

df_out = df_sales_order.rename(columns=needed_cols)[list(needed_cols.values())]

# Sort to group visually by WO, then by Product Number
df_out = df_out.sort_values(['WO', 'Product Number']).reset_index(drop=True)

# Usage:
final_sales_order = reorder_df_out_by_output(output_df, df_out)

final_sales_order['Product Number'] = final_sales_order['Product Number'].replace(mappings)

final_sales_order = final_sales_order.loc[:, ~final_sales_order.columns.duplicated()]

inventory_df.rename(columns={'Unnamed: 0': 'Part_Number', 'Available': 'Stock_Available'}, inplace=True)
# Upload to Google Sheet
google_cred_path = r"C:\Users\Admin\Downloads\pdfwo-466115-734096e1cef8.json"

if not final_sales_order.empty:
    write_df_to_gsheet(
        final_sales_order,
        sheet_name="PDF_WO",
        worksheet_name="Open Sales Order",
        cred_path=google_cred_path
    )
else:
    print("❌ No output to write.")


                 WO                    Product Number
0      TESTORDER123                                  
1       SO-20251118                     PCIe-PoE425bt
2       SO-20240013                                  
3       SO-20241091               AC-AR0147-H120-2203
4       SO-20241091               AC-AR0147-H190-2203
...             ...                               ...
13895   SO-20251170                      NRU-161V-AWP
13896   SO-20251170       GC-Jetson-NX16G-Orin-Nvidia
13897   SO-20251170     Adapter-Active-DP-HDMI-BENFEI
13898   SO-20251170  M.242-SSD-256GB-PCIe34-TLC5WT-TD
13899   SO-20251170                    AC-ISX031-H120

[13900 rows x 2 columns]
✅ Data written to Google Sheet: PDF_WO [Open Sales Order]


## Join Keys

In [472]:
# === Merge Picked Status into Sales Order ===
# Collapse to one row per WO_Number: Picked = True if ANY status == "Picked"
word_pick = (
    word_files_df
      .assign(Picked = word_files_df["status"].astype(str).str.strip().eq("Picked"))
      .groupby("WO_Number", as_index=False)["Picked"].max()
)

# Map True/False to desired labels
word_pick["Picked"] = word_pick["Picked"].map({True: "Picked", False: "No"})

# Merge
df_Order_Picked = (
    final_sales_order
      .merge(word_pick, left_on="WO", right_on="WO_Number", how="left")
      .drop(columns=["WO_Number"])
)
df_Order_Picked["Picked"] = df_Order_Picked["Picked"].fillna("No")


In [473]:
# 1) Picked qty per part
picked_parts = (df_Order_Picked.loc[df_Order_Picked["Picked"].eq("Picked")]
    .groupby("Product Number", as_index=False)["Qty"].sum()
    .rename(columns={"Product Number":"Part_Number","Qty":"Picked_Qty"}))

# 2) Inventory + picked
inv_plus = inventory_df.merge(picked_parts, on="Part_Number", how="left")
cols = ["On Hand","On Sales Order","On PO","Picked_Qty"]
inv_plus[cols] = inv_plus[cols].apply(pd.to_numeric, errors="coerce").fillna(0)

# 3) Merge WO lines(left join to keep all WOs)
structured_df = df_Order_Picked.merge(inv_plus, how="left", left_on="Product Number", right_on="Part_Number")
structured_df["Qty"] = pd.to_numeric(structured_df["Qty"], errors="coerce")
structured_df = structured_df.dropna(subset=["Qty"])

# 4) Constant ATP per part = On Hand - sum(assigned WO qty)  (assigned: Lead Time not 7/4 or 12/31)
structured_df["Lead Time"] = pd.to_datetime(structured_df["Lead Time"], errors="coerce").dt.normalize()
assigned_mask = ~(
    (structured_df["Lead Time"].dt.month.eq(7)  & structured_df["Lead Time"].dt.day.eq(4)) |
    (structured_df["Lead Time"].dt.month.eq(12) & structured_df["Lead Time"].dt.day.eq(31))
)
assigned_total = structured_df["Qty"].where(assigned_mask, 0).groupby(structured_df["Product Number"]).transform("sum")

structured_df["ATP"] = (structured_df["On Hand"] - assigned_total).clip(lower=0)
structured_df['In Stock'] = structured_df["On Hand"] - structured_df["Picked_Qty"]
structured_df["Component_Status"] = np.where(structured_df["ATP"] >= structured_df["Qty"], "Available", "Shortage")


structured_df.rename(columns={
    'In Stock': 'In Stock(Inventory)',
    'Stock_Available': 'Stock_Available(LT)',
    'ATP': 'ATP(LT)'
}, inplace=True)



structured_df

,Customer,Customer PO,WO,Product Number,Qty,Lead Time,Picked,Part_Number,Reorder Pt (Min),Max,...,Stock_Available(LT),Order,On PO,Reorder Qty,Next Deliv,Sales/Week,Picked_Qty,ATP(LT),In Stock(Inventory),Component_Status
0,"CoastIPC, Inc.",P96695,EO-20250002,RGS-8805GC,1.0,2025-12-31,No,RGS-8805GC,NaN,NaN,...,-1.0,NaN,0.0,0.0,NaN,0.1,0.0,0.0,0.0,Shortage
1,Noah Medical U.S.,PO208679,SO-20250326,E-2278GE,5.0,2025-09-11,Picked,E-2278GE,5.0,NaN,...,-2.0,NaN,8.0,0.0,08/27/2025,1.7,5.0,0.0,28.0,Shortage
2,Noah Medical U.S.,PO208679,SO-20250326,GC-RTXA4500-PNY,5.0,2025-09-11,Picked,GC-RTXA4500-PNY,NaN,NaN,...,6.0,NaN,0.0,0.0,NaN,0.7,5.0,6.0,32.0,Available
3,Noah Medical U.S.,PO208679,SO-20250326,DDR4-32GB-32-SM,10.0,2025-09-11,Picked,DDR4-32GB-32-SM,50.0,NaN,...,45.0,NaN,0.0,0.0,NaN,2.5,10.0,45.0,101.0,Available
4,Noah Medical U.S.,PO208679,SO-20250326,SSD-1920GB-TLC-IL1,10.0,2025-09-11,Picked,SSD-1920GB-TLC-IL1,NaN,NaN,...,-42.0,NaN,0.0,0.0,NaN,0.9,10.0,0.0,10.0,Shortage
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1029,"ACR-Advanced Construction Robotics, Inc.",PO_5769,SO-20251205,Wmkit-V-POC300_400,6.0,2025-12-31,No,Wmkit-V-POC300_400,7.0,99.0,...,21.0,NaN,0.0,0.0,NaN,1.1,0.0,27.0,30.0,Available
1030,LASERAX INC,00505553(1),SO-20251208,DDR4-16GB-32-SM,2.0,2025-12-31,No,DDR4-16GB-32-SM,50.0,NaN,...,10.0,NaN,260.0,0.0,08/27/2025,6.9,10.0,10.0,133.0,Available
1031,LASERAX INC,00505553(1),SO-20251208,M.280-SSD-256GB-SATA-TLC5WT-TD,2.0,2025-12-31,No,M.280-SSD-256GB-SATA-TLC5WT-TD,10.0,99.0,...,0.0,NaN,10.0,0.0,08/27/2025,0.5,1.0,0.0,0.0,Shortage
1032,LASERAX INC,00505553(1),SO-20251208,POC-410,2.0,2025-12-31,No,POC-410,10.0,NaN,...,9.0,X,0.0,2.0,NaN,1.8,0.0,9.0,9.0,Available


## Reports

### 1.By Sales Order

#### Most Cautious way is to assign parts Stock_Available > 0 (Pick ones that On Hand > On Sales Order)
#### Medium way is to assign parts ATP > 0 (Pick ones that On Hand > On Sales Order that has been assigned LT)
#### Most Aggressive way is to assign parts In Stock > 0 (Pick ones that can be found in Warehouse)

In [464]:
ERP_df= structured_df[['Customer', 'Customer PO', 'WO', 'Product Number', 'Qty', 
                               'Stock_Available(LT)', 'ATP(LT)', 'In Stock(Inventory)', 'On Hand', 'On Sales Order', 'On PO', 'Component_Status', 'Lead Time', 'Picked']]



ERP_df.loc[:, "Lead Time"] = pd.to_datetime(ERP_df["Lead Time"]).dt.date

filtered_df = ERP_df[ERP_df['WO'] == 'SO-20251182']

# Apply styling
styled = (
    filtered_df.style
    .map(lambda v: 'background-color: red; color: white' if v == 'Shortage' else '', subset='Component_Status')
    .set_table_styles([
            {
                'selector': 'th.col_heading.level0.col' + str(filtered_df.columns.get_loc('Stock_Available(LT)')),
                'props': [('background-color', 'orange'), ('color', 'black')]
            }
        ])
        .format(precision=0, na_rep='', thousands=",")
    )

# Display the styled DataFrame
display(styled)

,Customer,Customer PO,WO,Product Number,Qty,Stock_Available(LT),ATP(LT),In Stock(Inventory),On Hand,On Sales Order,On PO,Component_Status,Lead Time,Picked
975,Velasea,POV44817,SO-20251182,PA-280W-ET3,1,6,9,35,37,31,20,Available,2025-09-05 00:00:00,No
976,Velasea,POV44817,SO-20251182,Cbl-PC-TW-180CM1,1,47,8,185,283,236,100,Available,2025-09-05 00:00:00,No
977,Velasea,POV44817,SO-20251182,DINRAIL-O,1,-5,0,36,37,42,20,Shortage,2025-09-05 00:00:00,No
978,Velasea,POV44817,SO-20251182,Nuvo-9002LP-UL,1,2,3,4,4,2,0,Available,2025-09-05 00:00:00,No


##### Sort by Lead Time

In [466]:
Sorted_df = ERP_df.sort_values(by="Lead Time")
Sorted_df
Sorted_df.to_excel(r"C:\Users\Admin\OneDrive - neousys-tech\Desktop\Output.xlsx", sheet_name="Sheet3", index=False)

#### Ready for Production('In Stock' > 'Qty')

In [ ]:
# True for WOs where ALL lines have In Stock > 0 and not been picked
wo_ok = (
    ERP_df.groupby("WO")
    .apply(lambda g: (g["In Stock"] > g['Qty']).all() and (g["Picked"].eq("No").all()))
)


# 1) list of ready WOs
ready_wos = wo_ok[wo_ok].index.tolist()

# 2) full rows for those WOs
ready_df = ERP_df[ERP_df["WO"].isin(ready_wos)].copy()

print("Ready WOs:", ready_wos)


ready_df.to_excel(r"C:\Users\Admin\OneDrive - neousys-tech\Desktop\Output.xlsx", sheet_name="Sheet2", index=False)


Ready WOs: ['SO-20250329', 'SO-20250332', 'SO-20251067', 'SO-20251146', 'SO-20251160', 'SO-20251165', 'SO-20251182']


C:\Users\Admin\AppData\Local\Temp\ipykernel_22120\2035769883.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: (g["In Stock"] > g['Qty']).all() and (g["Picked"].eq("No").all()))


### 2.By Part Number

In [475]:
Inventory_Status = structured_df[['Product Number', 'Customer', 'WO', 'Qty', 'On Hand', 'On Sales Order', 'Stock_Available(LT)', 'ATP(LT)', 'In Stock(Inventory)','Lead Time', 'Picked']].sort_values(by="Lead Time", ascending=True)
Inventory_Status[Inventory_Status['Product Number'] == 'Cbl-PC-TW-180CM1']

,Product Number,Customer,WO,Qty,On Hand,On Sales Order,Stock_Available(LT),ATP(LT),In Stock(Inventory),Lead Time,Picked
97,Cbl-PC-TW-180CM1,Inficon Inc.,SO-20250600,21.0,283.0,236.0,47.0,53.0,230.0,2025-07-23,Picked
103,Cbl-PC-TW-180CM1,Inficon Inc.,SO-20250601,23.0,283.0,236.0,47.0,53.0,230.0,2025-07-23,Picked
150,Cbl-PC-TW-180CM1,Inficon Inc.,SO-20250728,30.0,283.0,236.0,47.0,53.0,230.0,2025-08-13,No
136,Cbl-PC-TW-180CM1,Inficon Inc.,SO-20250701,30.0,283.0,236.0,47.0,53.0,230.0,2025-08-13,No
143,Cbl-PC-TW-180CM1,Inficon Inc.,SO-20250726,30.0,283.0,236.0,47.0,53.0,230.0,2025-08-13,No
177,Cbl-PC-TW-180CM1,Scout AI Inc,SO-20250802,1.0,283.0,236.0,47.0,53.0,230.0,2025-08-20,Picked
245,Cbl-PC-TW-180CM1,CACI INC FEDERAL,SO-20250957,6.0,283.0,236.0,47.0,53.0,230.0,2025-08-20,Picked
274,Cbl-PC-TW-180CM1,Candela Technologies Inc.,SO-20251012,5.0,283.0,236.0,47.0,53.0,230.0,2025-08-27,No
534,Cbl-PC-TW-180CM1,Mission Control Space Services,SO-20251108,1.0,283.0,236.0,47.0,53.0,230.0,2025-08-27,Picked
672,Cbl-PC-TW-180CM1,Aerflite Canada Inc.,SO-20251140,1.0,283.0,236.0,47.0,53.0,230.0,2025-08-27,Picked


#### In Stock = On Hand - Picked Qty
#### Stock_Available = On Hand - On Sales Order
#### ATP = On Hand - Assigned LT Qty
#### Component_Status = np.where(structured_df["ATP"] >= structured_df["Qty"], "Available", "Shortage")



### 3.Inven